In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('/content/India_2017.csv')
print(data.head())

  LATITUDE LONGITUDE    PH   TH   CA     MG CHLORIDE SULPHATE NITRATE FLUORIDE
0  13.9728   77.7314  7.67  240   50  28.02      209       62      40     0.37
1  14.6717   77.4583  7.78  610   96  90.08      291       58      42     1.43
2  14.4117   77.7200  7.38  510  132  43.92      277       41      83     0.85
3  15.1167   77.6431  7.35  610  148  58.53      383       20      45     0.32
4  15.1681   77.3764  7.78  610  140  63.38      319      256     285     0.42


In [ ]:
# BIS limits -
# Ph = 6.5 to 8.5  NO Relaxation
# Th = 200 600
# CA = 75 200
# MG = 30 100
# Cl = 250 1000
# Sulphate = 200 400
# Nitrate = 75 NO Relaxation
# Flouride = 1 1.5
# TDS = 500 2000

In [ ]:
print(np.sum(data.isna()))
# print(data['CA'].value_counts())
# print(data['MG'].value_counts())
# print(data['FLUORIDE'].value_counts())

LATITUDE        4
LONGITUDE       4
PH              4
TH           1619
CA           1339
MG           1349
CHLORIDE        7
SULPHATE     1338
NITRATE      2136
FLUORIDE     1205
dtype: int64


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


In [ ]:
data['CA'] = data['CA'].str.strip()
data['MG'] = data['MG'].str.strip()
data['FLUORIDE'] = data['FLUORIDE'].str.strip()
data.replace(['ND', 'BDL'], 0, inplace=True)

In [ ]:
print(data['CA'].value_counts())

CA
40     505
48     354
20     349
44     348
36     325
      ... 
432      1
920      1
960      1
880      1
85       1
Name: count, Length: 558, dtype: int64


In [ ]:
exclude_columns = ['LATITUDE', 'LONGITUDE']
data[data.columns.difference(exclude_columns)] = data[data.columns.difference(exclude_columns)].apply(pd.to_numeric, errors='coerce')



In [ ]:
print(np.sum(data.isna()))

LATITUDE        4
LONGITUDE       4
PH              9
TH           1622
CA           1342
MG           1352
CHLORIDE       10
SULPHATE     1342
NITRATE      2163
FLUORIDE     1273
dtype: int64


In [ ]:
null_values = data[data['CHLORIDE'].isnull()]['CHLORIDE']
print(null_values)

920    NaN
5816   NaN
8448   NaN
8455   NaN
8485   NaN
8501   NaN
Name: CHLORIDE, dtype: float64


In [ ]:
data = data.dropna()
print(np.sum(data.isna()))

LATITUDE     0
LONGITUDE    0
PH           0
TH           0
CA           0
MG           0
CHLORIDE     0
SULPHATE     0
NITRATE      0
FLUORIDE     0
dtype: int64


In [ ]:
print(data['CA'].value_counts())

CA
40.00     316
48.00     238
44.00     233
80.00     228
56.00     200
         ... 
158.80      1
60.65       1
6.40        1
11.21       1
236.00      1
Name: count, Length: 363, dtype: int64


In [ ]:
print(data)

     LATITUDE LONGITUDE    PH     TH     CA     MG  CHLORIDE  SULPHATE  \
0     13.9728   77.7314  7.67  240.0   50.0  28.02     209.0      62.0   
1     14.6717   77.4583  7.78  610.0   96.0  90.08     291.0      58.0   
2     14.4117   77.7200  7.38  510.0  132.0  43.92     277.0      41.0   
3     15.1167   77.6431  7.35  610.0  148.0  58.53     383.0      20.0   
4     15.1681   77.3764  7.78  610.0  140.0  63.38     319.0     256.0   
...       ...       ...   ...    ...    ...    ...       ...       ...   
7454  25.7714   88.1875  8.59  120.0   18.0  18.00      57.0       0.0   
7455  25.7878   88.1894  8.89   85.0   20.0   8.00      32.0       0.0   
7456  25.6811   88.1133  8.66  130.0   22.0  18.00      32.0       0.0   
7457  25.7078   88.1172  8.86   95.0   18.0  12.00      27.0       0.0   
7458  25.6214   88.1083  8.79  165.0   22.0  27.00      45.0       0.0   

      NITRATE  FLUORIDE  
0        40.0      0.37  
1        42.0      1.43  
2        83.0      0.85  
3      

In [ ]:
data = data.reset_index(drop=True)


In [ ]:
data.to_csv('temp.csv', index=False)

In [ ]:
def generate_sub_indices(data, sub_indices):
  columns = ['TH', 'CA', 'MG', 'CHLORIDE', 'SULPHATE', 'FLUORIDE', 'TDS']
  acceptable_limits = {'TH':200,'CA':75,'MG':30,'CHLORIDE':250,'SULPHATE':200,'FLUORIDE':1,'TDS':500}
  permissible_limits = {'TH':600,'CA':200,'MG':100,'CHLORIDE':1000,'SULPHATE':400,'FLUORIDE':1.5,'TDS':2000}

  sub_indices = pd.DataFrame(index=data.index)

  sub_indices['PH'] = 0.0
  for i, value in enumerate(data['PH']):
    # print(f"Index: {i}, pH Value: {value}")  # Debug print
    if value >= 6.5 and value <= 8.5:
        sub_indices['PH'].at[i] = 100
        # print(f"Index: {i}, PH Value: {sub_indices['PH'].at[i]}")  # Debug print
    else:
        sub_indices['PH'].at[i] = 0



  for col in columns:
    sub_indices[col] = 0.0
    for i,value in enumerate(data[col]):
      if value <= acceptable_limits[col]:
        sub_indices[col].at[i] = 100
      elif value > permissible_limits[col]:
        sub_indices[col].at[i] = 0
      else:
        sub_indices[col].at[i] = (permissible_limits[col] - value)/(permissible_limits[col]-acceptable_limits[col])*100


  sub_indices['NITRATE'] = 0.0
  for i,value in enumerate(data['NITRATE']):
    if value <= 75:
      sub_indices['NITRATE'].at[i] = 100
    else :
      sub_indices['NITRATE'].at[i] = 0

  return sub_indices

In [ ]:
sub_indices = pd.DataFrame()
sub_indices = generate_sub_indices(data, sub_indices)
print(sub_indices)

         PH     TH     CA          MG    CHLORIDE  SULPHATE  FLUORIDE  NITRATE
0     100.0   90.0  100.0  100.000000  100.000000     100.0     100.0    100.0
1     100.0    0.0   83.2   14.171429   94.533333     100.0      14.0    100.0
2     100.0   22.5   54.4   80.114286   96.400000     100.0     100.0      0.0
3     100.0    0.0   41.6   59.242857   82.266667     100.0     100.0    100.0
4     100.0    0.0   48.0   52.314286   90.800000      72.0     100.0      0.0
...     ...    ...    ...         ...         ...       ...       ...      ...
7454    0.0  100.0  100.0  100.000000  100.000000     100.0     100.0    100.0
7455    0.0  100.0  100.0  100.000000  100.000000     100.0      60.0    100.0
7456    0.0  100.0  100.0  100.000000  100.000000     100.0     100.0    100.0
7457    0.0  100.0  100.0  100.000000  100.000000     100.0     100.0    100.0
7458    0.0  100.0  100.0  100.000000  100.000000     100.0     100.0    100.0

[7459 rows x 8 columns]


In [ ]:
sub_indices.to_csv('temp2.csv', index=False)

In [ ]:
print(sub_indices.shape)

(11103, 9)


In [ ]:
new_order = ['PH','TH','CA','MG','CHLORIDE','SULPHATE','NITRATE','FLUORIDE','TDS']
sub_indices = sub_indices[new_order]
print(sub_indices)

         PH     TH     CA          MG    CHLORIDE  SULPHATE  NITRATE  FLUORIDE
0     100.0   90.0  100.0  100.000000  100.000000     100.0    100.0     100.0
1     100.0    0.0   83.2   14.171429   94.533333     100.0    100.0      14.0
2     100.0   22.5   54.4   80.114286   96.400000     100.0      0.0     100.0
3     100.0    0.0   41.6   59.242857   82.266667     100.0    100.0     100.0
4     100.0    0.0   48.0   52.314286   90.800000      72.0      0.0     100.0
...     ...    ...    ...         ...         ...       ...      ...       ...
7454    0.0  100.0  100.0  100.000000  100.000000     100.0    100.0     100.0
7455    0.0  100.0  100.0  100.000000  100.000000     100.0    100.0      60.0
7456    0.0  100.0  100.0  100.000000  100.000000     100.0    100.0     100.0
7457    0.0  100.0  100.0  100.000000  100.000000     100.0    100.0     100.0
7458    0.0  100.0  100.0  100.000000  100.000000     100.0    100.0     100.0

[7459 rows x 8 columns]


In [ ]:
sub_indices['LATITUDE'] = data['LATITUDE']
sub_indices['LONGITUDE'] = data['LONGITUDE']
new_order = ['LATITUDE','LONGITUDE','PH','TH','CA','MG','CHLORIDE','SULPHATE','NITRATE','FLUORIDE','TDS']

sub_indices = sub_indices[new_order]
print(sub_indices)

     LATITUDE LONGITUDE     PH     TH     CA          MG    CHLORIDE  \
0     13.9728   77.7314  100.0   90.0  100.0  100.000000  100.000000   
1     14.6717   77.4583  100.0    0.0   83.2   14.171429   94.533333   
2     14.4117   77.7200  100.0   22.5   54.4   80.114286   96.400000   
3     15.1167   77.6431  100.0    0.0   41.6   59.242857   82.266667   
4     15.1681   77.3764  100.0    0.0   48.0   52.314286   90.800000   
...       ...       ...    ...    ...    ...         ...         ...   
7454  25.7714   88.1875    0.0  100.0  100.0  100.000000  100.000000   
7455  25.7878   88.1894    0.0  100.0  100.0  100.000000  100.000000   
7456  25.6811   88.1133    0.0  100.0  100.0  100.000000  100.000000   
7457  25.7078   88.1172    0.0  100.0  100.0  100.000000  100.000000   
7458  25.6214   88.1083    0.0  100.0  100.0  100.000000  100.000000   

      SULPHATE  NITRATE  FLUORIDE  
0        100.0    100.0     100.0  
1        100.0    100.0      14.0  
2        100.0      0.0    

In [ ]:
# Specify the columns you want to round
cols_to_round = ['PH', 'TH', 'CA', 'MG', 'CHLORIDE', 'SULPHATE', 'FLUORIDE', 'TDS', 'NITRATE']

# Round the specified columns to 2 decimal places
sub_indices[cols_to_round] = sub_indices[cols_to_round].round(2)

[0.03634713 0.66755045 0.03526272 0.06432972 0.03429169 0.02069839
 0.08058257 0.06093731 0.14412291]

In [ ]:
shap_weights = {'PH':0.03176855,
 'TH':0.58346045,
 'CA':0.03082074,
 'MG':0.05622624,
 'CHLORIDE':0.02997204,
 'SULPHATE':0.01809105,
 'NITRATE':0.07043174,
 'FLUORIDE':0.05326117,
 'TDS':0.12596802}


In [ ]:
cols = ['PH','TH','CA','MG','CHLORIDE','SULPHATE','NITRATE','FLUORIDE','TDS']

sub_indices['AWQI'] = sum(sub_indices[col] * shap_weights[col] for col in cols)
print(sub_indices)


     LATITUDE LONGITUDE     PH     TH     CA      MG  CHLORIDE  SULPHATE  \
0     13.9728   77.7314  100.0   90.0  100.0  100.00    100.00     100.0   
1     14.6717   77.4583  100.0    0.0   83.2   14.17     94.53     100.0   
2     14.4117   77.7200  100.0   22.5   54.4   80.11     96.40     100.0   
3     15.1167   77.6431  100.0    0.0   41.6   59.24     82.27     100.0   
4     15.1681   77.3764  100.0    0.0   48.0   52.31     90.80      72.0   
...       ...       ...    ...    ...    ...     ...       ...       ...   
7454  25.7714   88.1875    0.0  100.0  100.0  100.00    100.00     100.0   
7455  25.7878   88.1894    0.0  100.0  100.0  100.00    100.00     100.0   
7456  25.6811   88.1133    0.0  100.0  100.0  100.00    100.00     100.0   
7457  25.7078   88.1172    0.0  100.0  100.0  100.00    100.00     100.0   
7458  25.6214   88.1083    0.0  100.0  100.0  100.00    100.00     100.0   

      NITRATE  FLUORIDE       AWQI  
0       100.0     100.0  93.324494  
1       100.0

In [ ]:
sub_indices['AWQI'] = sub_indices['AWQI'].round(2)
print(sub_indices)

     LATITUDE LONGITUDE     PH     TH     CA      MG  CHLORIDE  SULPHATE  \
0     13.9728   77.7314  100.0   90.0  100.0  100.00    100.00     100.0   
1     14.6717   77.4583  100.0    0.0   83.2   14.17     94.53     100.0   
2     14.4117   77.7200  100.0   22.5   54.4   80.11     96.40     100.0   
3     15.1167   77.6431  100.0    0.0   41.6   59.24     82.27     100.0   
4     15.1681   77.3764  100.0    0.0   48.0   52.31     90.80      72.0   
...       ...       ...    ...    ...    ...     ...       ...       ...   
7454  25.7714   88.1875    0.0  100.0  100.0  100.00    100.00     100.0   
7455  25.7878   88.1894    0.0  100.0  100.0  100.00    100.00     100.0   
7456  25.6811   88.1133    0.0  100.0  100.0  100.00    100.00     100.0   
7457  25.7078   88.1172    0.0  100.0  100.0  100.00    100.00     100.0   
7458  25.6214   88.1083    0.0  100.0  100.0  100.00    100.00     100.0   

      NITRATE  FLUORIDE   AWQI  
0       100.0     100.0  93.32  
1       100.0      14

In [ ]:
sub_indices.to_csv('AWQI_India_2017.csv', index=False)